In [2]:
import os
import pandas as pd
from datetime import datetime

In [3]:
def to_date(df):
    df['data'] = df['ano'].astype(str) + '-' + df['mes'].apply(lambda x: '0' + str(x) if x < 10 else str(x))
    df['data'] = pd.to_datetime(df['data'], format='%Y-%m')

## Carregando e tratando dados da SAGI

In [15]:
files = [f for f in os.listdir("../../Data/Fam/") if os.path.isfile(os.path.join("../../Data/Fam/", f))]
total = len(files)
dfs=[]
for file in files:
    df = pd.read_csv(f"../../Data/Fam/{file}", header=0, index_col='Código')
    df['ano'] = df['Referência'].apply(lambda x: int(x.split('/')[1]))
    df['mes'] = df['Referência'].apply(lambda x: int(x.split('/')[0]))
    df.drop(columns=['Referência'], inplace=True)
    df.set_index(['UF','Unidade Territorial','ano','mes'], inplace=True)
    dfs.append(df)
    print('Arquivo lido. Restam', total-len(dfs), 'arquivos...')
df = pd.concat(dfs, axis=1)


Arquivo lido. Restam 5 arquivos...
Arquivo lido. Restam 4 arquivos...
Arquivo lido. Restam 3 arquivos...
Arquivo lido. Restam 2 arquivos...
Arquivo lido. Restam 1 arquivos...
Arquivo lido. Restam 0 arquivos...


In [16]:
df.rename(columns={
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_nao_benef_ext_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_benef_pobreza',
    'Famílias beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de extrema pobreza':'fam_benef_ext_pobreza',
    'Famílias não beneficiárias no Programa Bolsa Família/Programa Auxílio Brasil em situação de pobreza':'fam_nao_benef_pobreza',
    'Famílias inscritas no Cadastro Único em situação de extrema pobreza':'fam_inscritas_ext_pobreza',
    'Famílias inscritas no Cadastro Único em situação de pobreza':'fam_insc_pobreza',
    },
    inplace = True)

In [17]:
df['demanda_reprimida_total'] = df['fam_nao_benef_pobreza'] + df['fam_nao_benef_ext_pobreza']

In [18]:
uf = df.groupby(['ano','mes','UF']).sum()

In [19]:
br = df.groupby(['ano','mes']).sum()

In [20]:
df.reset_index(inplace=True)
uf.reset_index(inplace=True)
br.reset_index(inplace=True)

In [21]:
uf.columns

Index(['ano', 'mes', 'UF', 'fam_nao_benef_ext_pobreza', 'fam_benef_pobreza',
       'fam_benef_ext_pobreza', 'fam_nao_benef_pobreza',
       'fam_inscritas_ext_pobreza', 'fam_insc_pobreza',
       'demanda_reprimida_total'],
      dtype='object')

In [22]:
to_date(br)
to_date(uf)
to_date(df)

## Carregando e tratando dados do CECAD

In [5]:
siglas = pd.read_csv('../estados_siglas.csv', usecols=['Sigla', 'Estado'])
uf_pes_cecad = pd.read_csv('../../Data/Pessoas/CECAD/UF_CECAD.csv')
uf_fam_cecad = pd.read_csv('../../Data/Fam/CECAD/UF_CECAD.csv')
br_fam_cecad = pd.read_csv('../../Data/Fam/CECAD/BR_CECAD.csv')
br_pes_cecad = pd.read_csv('../../Data/Pessoas/CECAD/BR_CECAD.csv')

In [6]:
def rename_cols(df, unidade, UF):
    if UF:
        df.columns = ['semnome','UF', f'{unidade}_nao_benef_ext_pobreza',  f'{unidade}_nao_benef_pobreza', f'{unidade}_benef_ext_pobreza', f'{unidade}_benef_pobreza','sr1','sr2',f'{unidade}_inscritas_ext_pobreza', f'{unidade}_insc_pobreza', 'ano', 'mes']
        df.drop(columns=['semnome','sr1','sr2'], inplace=True)
    else:
        df.columns = ['semnome',f'{unidade}_nao_benef_ext_pobreza',  f'{unidade}_nao_benef_pobreza', f'{unidade}_benef_ext_pobreza', f'{unidade}_benef_pobreza','sr1','sr2',f'{unidade}_inscritas_ext_pobreza', f'{unidade}_insc_pobreza', 'ano', 'mes']
        df.drop(columns=['semnome','sr1','sr2'], inplace=True)
    return df

In [7]:
uf_pes_cecad = rename_cols(uf_pes_cecad, 'pessoas', UF=True)
uf_fam_cecad = rename_cols(uf_fam_cecad, 'fam', UF=True)
br_fam_cecad = rename_cols(br_fam_cecad, 'fam', UF=False)
br_pes_cecad = rename_cols(br_pes_cecad, 'pessoas', UF=False)

In [8]:
uf_cecad = pd.merge(uf_pes_cecad, uf_fam_cecad, on=['UF', 'ano', 'mes'])

In [9]:
uf_cecad['UF'] = uf_cecad['UF'].apply(lambda x: x.split('-')[0].strip())

In [10]:
br_cecad = pd.merge(br_pes_cecad, br_fam_cecad, on=['ano', 'mes'])

In [17]:
# uf_cecad = cast_to_int(uf_cecad)
# br_cecad = cast_to_int(br_cecad)

In [11]:
uf_cecad['demanda_reprimida_total'] = uf_cecad['fam_nao_benef_pobreza'] + uf_cecad['fam_nao_benef_ext_pobreza']
br_cecad['demanda_reprimida_total'] = br_cecad['fam_nao_benef_pobreza'] + br_cecad['fam_nao_benef_ext_pobreza']
uf_cecad['demanda_reprimida_pessoas'] = uf_cecad['pessoas_nao_benef_pobreza'] + uf_cecad['pessoas_nao_benef_ext_pobreza']
br_cecad['demanda_reprimida_pessoas'] = br_cecad['pessoas_nao_benef_pobreza'] + br_cecad['pessoas_nao_benef_ext_pobreza']

### Adicionando novas linhas referentes à coleta do CECAD ao dataframe da SAGI

In [23]:
uf = pd.concat([uf,uf_cecad]).merge(siglas, left_on='UF', right_on='Sigla', how='left')
uf.drop(columns=['Sigla'], inplace=True)

In [24]:
uf

,ano,mes,UF,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data,pessoas_nao_benef_ext_pobreza,pessoas_nao_benef_pobreza,pessoas_benef_ext_pobreza,pessoas_benef_pobreza,pessoas_inscritas_ext_pobreza,pessoas_insc_pobreza,demanda_reprimida_pessoas,Estado
0,2012,8,AC,0.0,0.0,0.0,0.0,66434.0,24358.0,0.0,2012-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACRE
1,2012,8,AL,0.0,0.0,0.0,0.0,417404.0,110517.0,0.0,2012-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ALAGOAS
2,2012,8,AM,0.0,0.0,0.0,0.0,317456.0,83948.0,0.0,2012-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMAZONAS
3,2012,8,AP,0.0,0.0,0.0,0.0,55886.0,16010.0,0.0,2012-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AMAPÁ
4,2012,8,BA,0.0,0.0,0.0,0.0,1831657.0,429469.0,0.0,2012-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BAHIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3181,2022,7,RR,8523.0,8514.0,53024.0,7138.0,61547.0,15652.0,15661.0,NaT,17525.0,14862.0,163764.0,31737.0,181289.0,46599.0,32387.0,RORAIMA
3182,2022,7,SC,28654.0,44489.0,148935.0,19063.0,177589.0,63552.0,47717.0,NaT,57338.0,46595.0,394432.0,157130.0,451770.0,203725.0,103933.0,SANTA CATARINA
3183,2022,7,SP,301769.0,361732.0,2004175.0,188503.0,2305944.0,550235.0,490272.0,NaT,585559.0,416880.0,4950785.0,1188523.0,5536344.0,1605403.0,1002439.0,SÃO PAULO
3184,2022,7,SE,23890.0,23195.0,362000.0,25549.0,385890.0,48744.0,49439.0,NaT,38454.0,36936.0,840992.0,73920.0,879446.0,110856.0,75390.0,SERGIPE


In [25]:
br = pd.concat([br,br_cecad])

In [26]:
dt = datetime.date(datetime.now())
br['ts'] = dt

In [27]:
to_date(br)
to_date(uf)

In [28]:
br.tail()

,ano,mes,fam_nao_benef_ext_pobreza,fam_benef_pobreza,fam_benef_ext_pobreza,fam_nao_benef_pobreza,fam_inscritas_ext_pobreza,fam_insc_pobreza,demanda_reprimida_total,data,pessoas_nao_benef_ext_pobreza,pessoas_nao_benef_pobreza,pessoas_benef_ext_pobreza,pessoas_benef_pobreza,pessoas_inscritas_ext_pobreza,pessoas_insc_pobreza,demanda_reprimida_pessoas,ts
113,2022,1,1188947.0,1805132.0,15787132.0,1081238.0,16975529.0,2886349.0,2270185.0,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-05
114,2022,2,1735253.0,1808270.0,15809665.0,1173835.0,17544358.0,2982086.0,2909088.0,2022-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-05
115,2022,3,1995898.0,1824711.0,15824769.0,1210277.0,17820102.0,3034968.0,3206175.0,2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-05
116,2022,4,2282276.0,1779297.0,15932367.0,1356972.0,18214051.0,3136245.0,3639248.0,2022-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-10-05
0,2022,7,1567764.0,2008976.0,17630042.0,1297764.0,19197806.0,3306740.0,2865528.0,2022-07-01,2867435.0,2437039.0,44828392.0,6736857.0,47695827.0,9173896.0,5304474.0,2022-10-05


In [23]:
br.to_parquet('Results/fam_br.parquet')
uf.to_parquet('Results/fam_uf.parquet')
df.to_parquet('Results/fam_mun.parquet')

In [24]:
br.to_parquet('Results/TEST/test_fam_br.parquet')
uf.to_parquet('Results/TEST/test_fam_uf.parquet')
df.to_parquet('Results/TEST/fam_mun.parquet')